# Extract Users to Parquet
Normalize `users.json` into analytics-ready Parquet datasets.

This notebook will:
- Read `data/raw/users.json` (fallback: `data/backup/users.json`)
- Build a Users table (exclude nested `projects` list)
- Build a User Badges table with one row per badge name per user
- Parse timestamps and add `scraped_at`
- Save to `data/users.parquet` and `data/user_badges.parquet`

In [1]:
# Imports
import json
from pathlib import Path
import pandas as pd

OUTPUT = Path('../../data')

user_path = '../../data/raw/users.json'
parquet_out = OUTPUT / 'users.parquet'

with open(user_path, 'r') as f:
    raw = json.load(f)

In [2]:
users = raw.get('users', [])
scraped_at = raw.get('scraped_at')

parse_dt = lambda s: pd.to_datetime(s, errors='coerce', utc=True) if pd.notna(s) else pd.NaT

user_cols = [
    'id','slack_id','display_name','bio','projects_count','devlogs_count','votes_count','ships_count',
    'coding_time_seconds','coding_time_seconds_today','balance','created_at','updated_at','avatar','custom_css'
]
user_records = []
for u in users:
    rec = {k: u.get(k) for k in user_cols}
    rec['created_at'] = parse_dt(rec.get('created_at'))
    rec['updated_at'] = parse_dt(rec.get('updated_at'))
    rec['scraped_at'] = parse_dt(scraped_at) if scraped_at else pd.Timestamp.utcnow()
    user_records.append(rec)

df_users = pd.DataFrame(user_records)
df_users

,id,slack_id,display_name,bio,projects_count,devlogs_count,votes_count,ships_count,coding_time_seconds,coding_time_seconds_today,balance,created_at,updated_at,avatar,custom_css,scraped_at
0,1,U03DFNYGPCN,Malted,None,1,1,12,1,125887,0,You need to have a pocket watcher badge to vie...,2025-06-15 16:27:55.496000+00:00,2025-08-10 00:54:37.885000+00:00,https://avatars.slack-edge.com/2025-04-02/8691...,None,2025-08-11 15:44:17.206000+00:00
1,2,U080A3QP42C,Rowan,we ball hard,2,0,2,0,587160,1813,You need to have a pocket watcher badge to vie...,2025-06-15 16:30:11.969000+00:00,2025-08-10 02:48:09.410000+00:00,https://avatars.slack-edge.com/2025-06-28/9118...,None,2025-08-11 15:44:17.206000+00:00
2,3,U06QK6AG3RD,nora,i'm like if a girl was working on Summer of Ma...,2,3,1,2,318710,0,You need to have a pocket watcher badge to vie...,2025-06-15 16:54:28.478000+00:00,2025-08-06 19:48:37.175000+00:00,https://avatars.slack-edge.com/2024-10-26/7934...,"body { filter: invert(1); font-family: ""Papyru...",2025-08-11 15:44:17.206000+00:00
3,4,U05F4B48GBF,cskartikey,uhhhh?,3,5,35,0,264024,6756,You need to have a pocket watcher badge to vie...,2025-06-15 17:35:55.246000+00:00,2025-07-09 15:02:58.138000+00:00,https://avatars.slack-edge.com/2025-05-17/8913...,body {\n display: none;\n}\n,2025-08-11 15:44:17.206000+00:00
4,5,U07L45W79E1,Neon,Meow meow - https://saahild.com,11,27,42,6,577233,0,You need to have a pocket watcher badge to vie...,2025-06-15 18:46:29.520000+00:00,2025-07-08 17:55:04.530000+00:00,https://avatars.slack-edge.com/2025-04-22/8794...,"body::after {\r\n content: """";\r\n position:...",2025-08-11 15:44:17.206000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21138,21224,U04J5CM3T46,Aaron D'Souza,None,0,0,2,0,0,0,You need to have a pocket watcher badge to vie...,2025-08-11 15:06:06.790000+00:00,2025-08-11 15:26:33.705000+00:00,https://secure.gravatar.com/avatar/bd61f30a8b8...,None,2025-08-11 15:44:17.206000+00:00
21139,21225,U04JC0N6ZC2,Brenden D'Souza,None,0,0,1,0,0,0,You need to have a pocket watcher badge to vie...,2025-08-11 15:06:06.985000+00:00,2025-08-11 15:27:55.245000+00:00,https://avatars.slack-edge.com/2023-01-23/4680...,None,2025-08-11 15:44:17.206000+00:00
21140,21226,U082RG5RML5,Sura,None,0,0,0,0,0,0,You need to have a pocket watcher badge to vie...,2025-08-11 15:12:14.751000+00:00,2025-08-11 15:13:46.145000+00:00,https://secure.gravatar.com/avatar/6e3088f2168...,None,2025-08-11 15:44:17.206000+00:00
21141,21227,U099W5VT830,Wilfrid,None,0,0,0,0,0,0,You need to have a pocket watcher badge to vie...,2025-08-11 15:31:57.902000+00:00,2025-08-11 15:31:57.902000+00:00,https://secure.gravatar.com/avatar/1b037236a40...,None,2025-08-11 15:44:17.206000+00:00


Add the shells data too

In [3]:
from pathlib import Path

shells_path = Path('../../data/raw/shells.json')
entries = []
if shells_path.exists():
    try:
        with open(shells_path, 'r') as f:
            shells_raw = json.load(f)
        if isinstance(shells_raw, list):
            entries = shells_raw
        elif isinstance(shells_raw, dict):
            entries = shells_raw.get('entries') or shells_raw.get('items') or []
        else:
            entries = []
    except Exception as e:
        print('Warning: failed to read shells.json:', e)
        entries = []
else:
    print('Warning: shells.json not found at', shells_path)
    entries = []

shells_map = {}
for e in entries:
    sid = e.get('slack_id')
    if not sid:
        continue
    try:
        shells_val = e.get('shells', 0)
        shells_map[sid] = int(shells_val) if shells_val is not None else 0
    except Exception:
        shells_map[sid] = 0

if not df_users.empty:
    df_users['balance'] = df_users['slack_id'].map(shells_map).fillna(0).astype('Int64')

(df_users[['id','slack_id','balance']].head(10) if 'balance' in df_users.columns else df_users.head(5))

,id,slack_id,balance
0,1,U03DFNYGPCN,0
1,2,U080A3QP42C,3
2,3,U06QK6AG3RD,385
3,4,U05F4B48GBF,239
4,5,U07L45W79E1,365
5,6,U0C7B14Q3,831
6,7,U082DPCGPST,1
7,8,U091BS3FQ83,0
8,9,U0927DMTR9N,0
9,10,U088V2C21AB,21


In [4]:
badge_records = []
for u in users:
    uid = u.get('id')
    for b in u.get('badges', []) or []:
        badge_records.append({
            'user_id': uid,
            'badge_name': b.get('name'),
            'badge_text': b.get('text'),
            'badge_icon': b.get('icon'),
            'scraped_at': parse_dt(scraped_at) if scraped_at else pd.Timestamp.utcnow(),
        })

df_badges = pd.DataFrame(badge_records)

In [5]:
p_users = OUTPUT / 'users.parquet'
p_badges = OUTPUT / 'user_badges.parquet'

if not df_users.empty:
    df_users.to_parquet(p_users, engine="pyarrow", index=False)
if not df_badges.empty:
    df_badges.to_parquet(p_badges, engine="pyarrow", index=False)

print('Saved:')
for name, p in {'users': p_users, 'user_badges': p_badges}.items():
    if p.exists():
        print(f' - {name}: {p}')

print('\nRow counts:', {'users': len(df_users), 'user_badges': len(df_badges)})

Saved:
 - users: ../../data/users.parquet
 - user_badges: ../../data/user_badges.parquet

Row counts: {'users': 21143, 'user_badges': 2157}


In [6]:
df_users

,id,slack_id,display_name,bio,projects_count,devlogs_count,votes_count,ships_count,coding_time_seconds,coding_time_seconds_today,balance,created_at,updated_at,avatar,custom_css,scraped_at
0,1,U03DFNYGPCN,Malted,None,1,1,12,1,125887,0,0,2025-06-15 16:27:55.496000+00:00,2025-08-10 00:54:37.885000+00:00,https://avatars.slack-edge.com/2025-04-02/8691...,None,2025-08-11 15:44:17.206000+00:00
1,2,U080A3QP42C,Rowan,we ball hard,2,0,2,0,587160,1813,3,2025-06-15 16:30:11.969000+00:00,2025-08-10 02:48:09.410000+00:00,https://avatars.slack-edge.com/2025-06-28/9118...,None,2025-08-11 15:44:17.206000+00:00
2,3,U06QK6AG3RD,nora,i'm like if a girl was working on Summer of Ma...,2,3,1,2,318710,0,385,2025-06-15 16:54:28.478000+00:00,2025-08-06 19:48:37.175000+00:00,https://avatars.slack-edge.com/2024-10-26/7934...,"body { filter: invert(1); font-family: ""Papyru...",2025-08-11 15:44:17.206000+00:00
3,4,U05F4B48GBF,cskartikey,uhhhh?,3,5,35,0,264024,6756,239,2025-06-15 17:35:55.246000+00:00,2025-07-09 15:02:58.138000+00:00,https://avatars.slack-edge.com/2025-05-17/8913...,body {\n display: none;\n}\n,2025-08-11 15:44:17.206000+00:00
4,5,U07L45W79E1,Neon,Meow meow - https://saahild.com,11,27,42,6,577233,0,365,2025-06-15 18:46:29.520000+00:00,2025-07-08 17:55:04.530000+00:00,https://avatars.slack-edge.com/2025-04-22/8794...,"body::after {\r\n content: """";\r\n position:...",2025-08-11 15:44:17.206000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21138,21224,U04J5CM3T46,Aaron D'Souza,None,0,0,2,0,0,0,0,2025-08-11 15:06:06.790000+00:00,2025-08-11 15:26:33.705000+00:00,https://secure.gravatar.com/avatar/bd61f30a8b8...,None,2025-08-11 15:44:17.206000+00:00
21139,21225,U04JC0N6ZC2,Brenden D'Souza,None,0,0,1,0,0,0,0,2025-08-11 15:06:06.985000+00:00,2025-08-11 15:27:55.245000+00:00,https://avatars.slack-edge.com/2023-01-23/4680...,None,2025-08-11 15:44:17.206000+00:00
21140,21226,U082RG5RML5,Sura,None,0,0,0,0,0,0,0,2025-08-11 15:12:14.751000+00:00,2025-08-11 15:13:46.145000+00:00,https://secure.gravatar.com/avatar/6e3088f2168...,None,2025-08-11 15:44:17.206000+00:00
21141,21227,U099W5VT830,Wilfrid,None,0,0,0,0,0,0,0,2025-08-11 15:31:57.902000+00:00,2025-08-11 15:31:57.902000+00:00,https://secure.gravatar.com/avatar/1b037236a40...,None,2025-08-11 15:44:17.206000+00:00
